<a href="https://colab.research.google.com/github/futureCodersSE/data-roles/blob/main/Data_jobs_reed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This finds all the job codes of a search of Data - Kent - radius=10 on Reed.co.uk

In [ ]:
x = np.array([1,2,3])
a = np.array([3,2,1])

x = np.append(x,[a])
print(x)

[1 2 3 3 2 1]


In [13]:
### My reed api_key: 96a7ec49-549c-4529-b9d2-fa3059a437b3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


#pages = np.range(63)

#url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=63&proximity=30"
url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"

def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def find_jobs_iterate(url):
  site_html = get_html(url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  jobs_found = results.find_all(class_="job-result-card")

  for job in jobs_found:
      job_id = int(job["id"].split("jobSection")[1])
      
      id_list = np.append(id_list,[job_id])
  return id_list
  
def find_total_pages(site_html):
  of_total_jobs = site_html.find(class_ = "search-results-footer").text
  #print(of_total_jobs)
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]
  total_jobs = total_jobs_split1.split("of ")[1]
  print(total_jobs)
#not used

#soup = get_html(url)
#id_list = find_jobs(soup)

#print(len(id_list))

In [14]:
from pandas.core.arrays.sparse import dtype
initial_url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"


def the_scrape(url):
  soup = get_html(url)
  results = soup.find(class_="col-sm-8 col-md-9 results-container")

  total_id_list = np.array([],dtype=int)
  jobs_found = results.find_all(class_="job-result-card")
  #print(jobs_found)
  for job in jobs_found:
      job_id = int(job["id"].split("jobSection")[1])
      
      total_id_list = np.append(total_id_list, [job_id])              
  print(total_id_list)


  of_total_jobs = results.find(class_ = "search-results-footer").text       #
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]                       #Finds the total number of jobs 
  total_jobs_str = total_jobs_split1.split("of ")[1]                        #and total number of pages needed to run
  total_jobs_list = total_jobs_str.split(",")
  total_pages = int(np.ceil(int("".join(map(str,total_jobs_list)) )/25))

  
  print("total pages: ", total_pages)
  print("total jobs: ", total_jobs_split1)
  print("1")
  for page_no in np.arange(2,total_pages+1):
    print(page_no)
    url2 = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno="+str(page_no)+"&proximity=30"
    print(url2)
    page_id_list = find_jobs_iterate(url2)

    total_id_list = np.append(total_id_list,[page_id_list])
  return total_id_list

id_list = the_scrape(initial_url)
id_list = id_list.astype("int")
#print(id_list)



[48653746 48653746 48631725 48529572 48223150 48320718 48211407 48648003
 48465441 48636996 48641144 48350103 48630045 48574900 48335088 48315303
 48316096 48610553 48586852 48389352 48535395 48431584 48561273 48264714
 48475959 48405690]
total pages:  59
total jobs:  





                        1 - 25 of 1,469
1
2
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=2&proximity=30
3
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=3&proximity=30
4
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=4&proximity=30
5
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=5&proximity=30
6
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=6&proximity=30
7
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=7&proximity=30
8
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=8&proximity=30
9
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=9&proximity=30
10
https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pa

In [15]:
print(len(id_list))

1528


In [ ]:
def call_api2():
  url = "https://www.reed.co.uk/api/1.0/search?keywords=Data&locationname=ashford&distancefromlocation=30&postedbydirectemployer=false" # #&postedbyrecruitmentagency=true
  job_df = pd.DataFrame()
  post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
  json_data = post_request.json()["results"]


  new_row = pd.json_normalize(json_data)
  job_df = job_df.append(new_row,ignore_index=True)
  job_df = job_df[job_df["jobTitle"].str.contains("Data")]
  return job_df

temp_df = call_api2()
display(temp_df.drop(columns=["jobId","employerId","employerProfileId","employerProfileName","applications","jobDescription"]))

In [17]:
def call_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  url = ""
  job_df = pd.DataFrame()


  for job_id in id_list:
    url = base_url+str(job_id)
    post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
    json_data = post_request.json()


    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df
full_job_df = call_api(id_list)
display(full_job_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   employerId           1528 non-null   int64 
 1   employerName         1528 non-null   object
 2   jobId                1528 non-null   int64 
 3   jobTitle             1528 non-null   object
 4   locationName         1528 non-null   object
 5   minimumSalary        1268 non-null   object
 6   maximumSalary        1268 non-null   object
 7   yearlyMinimumSalary  1268 non-null   object
 8   yearlyMaximumSalary  1268 non-null   object
 9   currency             1268 non-null   object
 10  salaryType           1528 non-null   object
 11  salary               1268 non-null   object
 12  datePosted           1528 non-null   object
 13  expirationDate       1528 non-null   object
 14  externalUrl          292 non-null    object
 15  jobUrl               1528 non-null   object
 16  partTi

None

In [19]:
#print(full_job_df.columns)

job_df = full_job_df.drop(columns = ['employerId','minimumSalary', 'maximumSalary', 'yearlyMinimumSalary',
       'yearlyMaximumSalary', 'currency', 'salaryType', 'datePosted',
       'expirationDate','partTime', 'fullTime', 'contractType', 'jobDescription', 'applicationCount'])

display(job_df)

,employerName,jobId,jobTitle,locationName,salary,externalUrl,jobUrl
0,Veolia UK,48653746,Data Architect,United Kingdom,"£65,000 - £84,600 per annum, inc benefits",https://veoliauki.wd3.myworkdayjobs.com/en-US/...,https://www.reed.co.uk/jobs/data-architect/486...
1,Veolia UK,48653746,Data Architect,United Kingdom,"£65,000 - £84,600 per annum, inc benefits",https://veoliauki.wd3.myworkdayjobs.com/en-US/...,https://www.reed.co.uk/jobs/data-architect/486...
2,Premier IT,48631725,Data Analyst,Basildon,"£30,000 - £35,000 per annum",None,https://www.reed.co.uk/jobs/data-analyst/48631725
3,Commercial Services Interim & Executive Search,48529572,Data Analyst,West Malling,"£41,216 - £47,199 per annum",None,https://www.reed.co.uk/jobs/data-analyst/48529572
4,Morgan Sindall Property Services,48223150,Data Analyst,Billericay,"£26,000 - £34,000 per annum, inc benefits",https://morgansindall.octo-firstclass.co.uk/ca...,https://www.reed.co.uk/jobs/data-analyst/48223150
...,...,...,...,...,...,...,...
1523,FirstPort,48421098,Residential Development Manager Ref 1025,Folkestone,"£17,290 per annum",https://ce0511li.webitrent.com/ce0511li_webrec...,https://www.reed.co.uk/jobs/residential-develo...
1524,SAGA,48691434,Business Analyst,Sandgate,None,https://careershub.saga.co.uk/members/?j=2141&...,https://www.reed.co.uk/jobs/business-analyst/4...
1525,FirstPort,48460519,Assistant Estate Manager Ref 1053,Edenbridge,"£2,953 per annum",https://ce0511li.webitrent.com/ce0511li_webrec...,https://www.reed.co.uk/jobs/assistant-estate-m...
1526,FirstPort,48460518,Assistant Estate Manager Ref 1052,Edenbridge,"£4,134 per annum",https://ce0511li.webitrent.com/ce0511li_webrec...,https://www.reed.co.uk/jobs/assistant-estate-m...


In [ ]:
#@title Default title text
###Failed
"""
key = "96a7ec49-549c-4529-b9d2-fa3059a437b3"

exm_url = """https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&loc 
ationName={locationName}&employerId={employerId}&distanceFromLocation={distance in miles}"""


url = "https://www.reed.co.uk/api/1.0/search?keywords=data&location=ashfordkent&distanceFromLocation=10" #

post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']

if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)

json_keys = json_data.keys()
print("total results found: ",json_data["totalResults"])
#'results' has the top 100 results from the api
#'ambiguousLocations' is empty
#'totalResults' is: 43984, the total value found by the api
print(json_keys) #.keys()
2
results_df = pd.json_normalize(json_data["results"]) 
#ambi_df = pd.json_normalize(json_data["ambiguousLocations"])
display(results_df.info())

results_df.drop(columns = ["minimumSalary", 'currency', 'expirationDate', 'date','jobDescription',
       'applications',"employerProfileName","employerProfileId","maximumSalary"],inplace=True)


def outside_kent():
  drop_cities = ["Bristol", "Manchester","Colchester","Reading","Brussels","Glasgow","Edinburgh","Southampton","Oxford","Cambridge"]
  for city in drop_cities:
    results_df.drop(results_df[results_df["locationName"] == city].index, inplace =True)
#outside_kent()

#results_df.drop("locationName")
display(results_df)
less_results_df = results_df[['employerName', 'jobTitle' ,'locationName']]
#display(less_results_df)
"""

In [ ]:
###api test

job_id = "48445740"
print(job_id)
url = "https://www.reed.co.uk/api/1.0/jobs/" # 48401296

url = url+job_id
post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']
if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)
print(json_data)

df = pd.json_normalize(json_data)
df




48445740
connection made, code:  200
{'employerId': 408731, 'employerName': 'International SOS', 'jobId': 48445740, 'jobTitle': 'Data Analyst', 'locationName': 'London', 'minimumSalary': None, 'maximumSalary': None, 'yearlyMinimumSalary': None, 'yearlyMaximumSalary': None, 'currency': None, 'salaryType': 'per annum', 'salary': None, 'datePosted': '07/10/2022', 'expirationDate': '06/11/2022', 'externalUrl': 'https://international-sos-pte-ltd.contactrh.com/jobs/5114/40712131', 'jobUrl': 'https://www.reed.co.uk/jobs/data-analyst/48445740', 'partTime': False, 'fullTime': True, 'contractType': 'Permanent', 'jobDescription': ' <p>Company description:</p> <p>International SOS is the world’s leading medical and security services company with over 12,000 employees working in 1,000 locations in 90 countries.&nbsp;International SOS Government Services administers the TRICARE Overseas Program providing high-quality, accessible  health care services for U.S. Military service members, family members

,employerId,employerName,jobId,jobTitle,locationName,minimumSalary,maximumSalary,yearlyMinimumSalary,yearlyMaximumSalary,currency,...,salary,datePosted,expirationDate,externalUrl,jobUrl,partTime,fullTime,contractType,jobDescription,applicationCount
0,408731,International SOS,48445740,Data Analyst,London,None,None,None,None,None,...,None,07/10/2022,06/11/2022,https://international-sos-pte-ltd.contactrh.co...,https://www.reed.co.uk/jobs/data-analyst/48445740,False,True,Permanent,<p>Company description:</p> <p>International ...,22
